In [5]:
%pip install pandas scikit-learn graphviz

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl (40.9 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from scripts import clean_data
%pip install pandas scikit-learn graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from graphviz import Source
from IPython.display import display

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load the data
df = pd.read_csv("./data/clean_data.csv")
X = df
T = pd.read_csv("./data/cleaned_data_combined_modified.csv")["Label"]

# Split the data into training and testing sets
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2, random_state=42)

# Split the data into training and validation sets
X_train, X_valid, T_train, T_valid = train_test_split(X_train, T_train, test_size=0.2, random_state=42)


# Optimal hyperparameters
# model = DecisionTreeClassifier()
# model.fit(X_train, T_train)

def build_all_models(max_depths,
                     min_samples_split,
                     criterion,
                     X_train=X_train,
                     t_train=T_train,
                     X_valid=X_valid,
                     t_valid=T_valid):
    """
    Parameters:
        `max_depths` - A list of values representing the max_depth values to be
                       try as hyperparameter values
        `min_samples_split` - An list of values representing the min_samples_split
                       values to try as hyperpareameter values
        `criterion` -  A string; either "entropy" or "gini"

    Returns a dictionary, `out`, whose keys are the the hyperparameter choices, and whose values are
    the training and validation accuracies (via the `score()` method).
    In other words, out[(max_depth, min_samples_split)]['val'] = validation score and
                    out[(max_depth, min_samples_split)]['train'] = training score
    For that combination of (max_depth, min_samples_split) hyperparameters.
    """
    out = {}

    for d in max_depths:
        for s in min_samples_split:
            out[(d, s)] = {}
            # Create a DecisionTreeClassifier based on the given hyperparameters and fit it to the data
            tree = DecisionTreeClassifier(criterion=criterion, min_samples_split=s, max_depth=d)
            tree.fit(X_train, T_train)
            out[(d, s)]['val'] = tree.score(X_valid, t_valid)
            out[(d, s)]['train'] = tree.score(X_train, t_train)
    return out


max_depths = [1, 5, 10, 15, 20, 25, 30, 50, 100]
min_samples_split = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]

res = build_all_models(max_depths=max_depths,min_samples_split=min_samples_split,criterion="entropy",X_train=X_train,t_train=T_train,X_valid=X_valid,t_valid=T_valid) # run build_all_models on all hyperparams
optimal=(-1,-1)
best_score=0
for d, s in res:
    if res[(d, s)]['val'] > best_score:
        best_score = res[(d, s)]['val']
        optimal = (d, s)
print("Best parameters: {}".format(optimal))
print("Best score: {}".format(best_score))


model = DecisionTreeClassifier(criterion="entropy", min_samples_split=optimal[1], max_depth=optimal[0])
model.fit(X_train, T_train)
print(model.score(X_train, T_train))
print(model.score(X_valid, T_valid))
print(model.score(X_test, T_test))
dot_data = tree.export_graphviz(model)
graph = Source(dot_data)
# graph.render("decision_tree", format="png", cleanup=True)  # Saves as decision_tree.png


NameError: name 'X_train' is not defined